In [3]:
import pandas as pd #dataframes

In [4]:
#create a dataframe for the months needed after importing the files
df = pd.DataFrame()
for month in ['03','04','05','06']:
    df = df.append(pd.read_csv("original_data/yellow_tripdata_2019-"+month+".csv" , low_memory = False), ignore_index=True)

#rename the location id to match the taxi zone location id    
df.rename(columns={'PULocationID':'LocationID'}, inplace=True)
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'], format='%Y-%m-%d %H:%M:%S')

In [5]:
#create a new df from selected columns
df2 = df[['LocationID', 'tpep_pickup_datetime', 'passenger_count']]

#change the datetime to date
df2['tpep_pickup_datetime'] = df2['tpep_pickup_datetime'].dt.date
yellow_taxi_2019=df2[['tpep_pickup_datetime', 'passenger_count']]
yellow_taxi_2019.head()

<ipython-input-5-a65fe770ea84>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['tpep_pickup_datetime'] = df2['tpep_pickup_datetime'].dt.date


,tpep_pickup_datetime,passenger_count
0,2019-03-01,1
1,2019-03-01,2
2,2019-03-01,1
3,2019-03-01,1
4,2019-03-01,1


In [7]:
#create a dataframe by merging with taxi zone
zone_lookup = pd.read_csv("original_data/taxi_zone_lookup.csv") # this dataset have a map between the LocationID and Borough
yellow_taxi_2019_boroughs = df2.merge(zone_lookup[['LocationID', 'Borough']], how='inner' ,on='LocationID').fillna(0)

#drop unnecessary column
yellow_taxi_2019_boroughs = yellow_taxi_2019_boroughs.drop(['LocationID'], axis=1)
yellow_taxi_2019_boroughs.head()

,tpep_pickup_datetime,passenger_count,Borough
0,2019-03-01,1,Queens
1,2019-03-01,2,Queens
2,2019-03-01,1,Queens
3,2019-03-01,1,Queens
4,2019-03-01,4,Queens


In [15]:
#create a dataframe with only usage
yellow_2019= pd.DataFrame(df2.tpep_pickup_datetime.value_counts().reset_index().values, columns=["Date", "Yellow_Taxi_2019"])
yellow_2019 = yellow_2019.sort_values(by="Date")
yellow_2019

,Date,Yellow_Taxi_2019
158,2001-01-01,2
161,2002-12-31,2
152,2008-08-08,3
124,2008-12-31,52
123,2009-01-01,148
...,...,...
165,2019-11-21,1
159,2019-12-12,2
173,2020-01-03,1
146,2033-04-27,3


In [16]:
#save the dataframe as csv file
yellow_2019.to_csv("yellow_2019.csv")

In [17]:
#create a dataframe to get the usage by borough
yellow_2019_borough = yellow_taxi_2019_boroughs.groupby("Borough").count().reset_index()
yellow_2019_borough.rename(columns={'tpep_pickup_datetime':'yellow_taxi_2019'}, inplace=True)
yellow_2019_borough=yellow_2019_borough.drop(['passenger_count'], axis=1)
yellow_2019_borough

,Borough,yellow_taxi_2019
0,Bronx,52103
1,Brooklyn,367051
2,EWR,2785
3,Manhattan,26999017
4,Queens,2086686
5,Staten Island,1161
6,Unknown,263166


In [18]:
#save the dataframe as csv file
yellow_2019_borough.to_csv("yellow_2019_borough.csv")